<a href="https://colab.research.google.com/github/JackHsuan/LineBotProject/blob/master/LineBot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk
!pip install pymysql
!pip install selenium
!apt-get update #to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install pyngrok
!pip install flask-ngrok #安裝Ngrok套件
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from google.colab import drive
import os
if(not(os.path.exists('/content/drive'))):
    drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/LineBotProject/')


In [ ]:
# -*- coding: utf-8 -*-
from flask import Flask, request, abort
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent ,UnfollowEvent ,FollowEvent, PostbackEvent, ImageMessage, VideoMessage,QuickReply,QuickReplyButton,FlexSendMessage,
    TextMessage, TextSendMessage,ImageSendMessage, TemplateSendMessage, ButtonsTemplate, MessageAction, URIAction, PostbackAction,LocationAction,LocationMessage
)
import pymysql
import configparser
import requests
from bs4 import BeautifulSoup
import re
import urllib
import threading
from functools import partial
import requests.packages.urllib3
from flask_ngrok import run_with_ngrok#引用ngrok套件
from openpyxl import load_workbook
import json
import pandas as pd
import time
import concurrent.futures
# import os

requests.packages.urllib3.disable_warnings()

In [ ]:
# !export FLASK_ENV=development

In [ ]:
# def init_webhooks(base_url):
#     # Update inbound traffic via APIs to use the public-facing ngrok URL
#     pass

# def create_app():
#     app = Flask(__name__)

#     # Initialize our ngrok settings into Flask
#     app.config.from_mapping(
#         BASE_URL="http://localhost:5000",
#         USE_NGROK=os.environ.get("USE_NGROK", "False") == "True" and os.environ.get("WERKZEUG_RUN_MAIN") != "true"
#     )

#     if app.config.get("ENV") == "development" and app.config["USE_NGROK"]:
#         # pyngrok will only be installed, and should only ever be initialized, in a dev environment
#         from pyngrok import ngrok
#         from pyngrok.conf import PyngrokConfig
#         # Get the dev server port (defaults to 5000 for Flask, can be overridden with `--port`
#         # when starting the server
#         port = sys.argv[sys.argv.index("--port") + 1] if "--port" in sys.argv else 5000

#         # Open a ngrok tunnel to the dev server
#         pyngrok_config = PyngrokConfig(region="ap")

#         public_url = ngrok.connect(port,pyngrok_config=pyngrok_config)
#         print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, port))

#         # Update any base URLs or webhooks to use the public ngrok URL
#         app.config["BASE_URL"] = public_url
#         init_webhooks(public_url)

#     # ... Initialize Blueprints and the rest of our app

#     return app

In [ ]:
# app = create_app()
# app.run

In [ ]:
app = Flask(__name__)
config = configparser.ConfigParser()
config.read("./config.ini")
RichMenuId = config['line_bot']['RichMenuId']
file_path = '/content/drive/My Drive/forMac/jupyter_notebook_root/AI_class/jupyter_root_temp/Jack_line/LineBot_project'
job_item_Message_Json = json.load(open('LineBotJsonFile/job_list_Message.json',))
line_bot_api = LineBotApi(config['line_bot']['Channel_Access_Token'])
handler = WebhookHandler(config['line_bot']['Channel_Secret'])
_104_icon_url = 'https://static.104.com.tw/logo/104logo_o_180x26.svg'


In [ ]:
# _111_icon_url = ''

In [ ]:
# df = Catch_104_joblist()

In [ ]:
# job_item_Message_Json['contents']['body']['contents'][0]['contents'][1]['action']['uri']

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     future = executor.submit(foo, 'world!')
#     return_value = future.result()
#     print(return_value)

In [ ]:
# params = {'ro': '0', 'keyword': 'Python', 'isnew': '30', 'mode': 's', 'area': '', 'asc': '0', 'order': '11', 'jobsource': '2018indexpoc', 'page': '3', 'jobexp': '1'}

# joblist = Catch_104_joblist(my_paramas = params)

In [ ]:
# print(joblist)

In [ ]:
# job_data_frames_old = pd.read_excel('jobs.xlsx',sheet_name="newsSearch")
# # print(job_data_frames_old)

# new_job_data_frames = job_data_frames_old.append(joblist,ignore_index = True,sort = False)
# print(new_job_data_frames)
# # new_job_data_frames.drop_duplicates(inplace = True)
# # print(new_job_data_frames)
# # new_job_data_frames.to_excel('./jobs.xlsx',sheet_name = sheetName,startrow=1, index = False,header= False)
# #     # writer.save()
# # print("Write Done")

In [ ]:
# new_job_data_frames.drop_duplicates(subset=['公司名稱', '工作職稱', '工作內容'],inplace = True)
# print(new_job_data_frames)

In [ ]:
def Catch_104_joblist(my_paramas="",keyword="",Jobtype='全職',area='台北市',page=1):
    # 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    area_encode={'台北市':'600100100',
        '新北市':'6001002000',
        '宜蘭縣':'6001003000',
        '基隆市':'6001004000',
        '桃園市':'6001005000',
        '新竹縣市':'6001006000',
        '苗栗縣':'6001007000',
        '台中市':'6001008000',
        '彰化縣':'6001010000',
        '南投縣':'6001011000',
        '雲林縣':'6001012000',  
        '嘉義縣市':'6001013000',  
        '台南市':'6001014000',  
        '高雄市':'6001016000',  
        '屏東縣':'6001018000',  
        '台東縣':'6001019000',  
        '花蓮縣':'6001020000',  
        '澎湖縣':'6001021000',  
        '金門縣':'6001022000',  
        '連江縣':'6001020000',}
    job_type_decode = ['全部','全職','兼職','高階','派遣','接案','家教','其他']
    # 查詢的關鍵字
    if (my_paramas ==""):
        my_params = {'ro':str(int(job_type_decode.index(str(Jobtype)))), # 限定全職的工作，如果不限定則輸入0
                    'keyword':keyword, # 想要查詢的關鍵字
                    'area':area_encode[area], # 限定在台北的工作
                    'isnew':'30', # 只要最近一個月有更新的過的職缺
                    'mode':'s',
                    'page':int(page)} # 清單的瀏覽模式
        sheetName = "personalSearch"
    else:
        sheetName = "newsSearch"
        my_params = my_paramas
    print("in Catch_104_joblist")
    print(my_params)
    index = requests.get('https://www.104.com.tw/jobs/search/?' , params = my_params, headers = headers)

    # 透過BeautifulSoup解析資料
    soup = BeautifulSoup(index.text, 'html.parser')
    List = soup.findAll('a',{'class':'js-job-link'})
    print('共有 ' + str(len(List)) + ' 筆資料')
    print(List)
    def bind(cate):
        k = []
        for i in cate:
            if len(i.text) > 0:
                k.append(i.text)
        return str(k)
    job_type_list = ['全職','兼職','高階','派遣','接案','家教','其他']
    JobList = pd.DataFrame()

    i = 0
    for i in range(9):
        # print('正在處理第' + str(i) + '筆，共 ' + str(len(List)) + ' 筆資料')
        content = List[i]
        # print(content)
        # 這裡用Try的原因是，有時候爬太快會遭到系統阻擋導致失敗。因此透過這個方式，當我們遇到錯誤時，會重新再爬一次資料！
        # try:
        job_num = content.attrs['href'].strip('//').split('?')[0].split('/')[-1]
        url = 'https://www.104.com.tw/job/ajax/content/%s'%job_num
        headers = {
            "Referer": "https://www.104.com.tw/job/%s"%job_num,
        }
        response = requests.get(url = url, headers = headers)
        response.encoding = "utf-8"
        print(response.text)
        try:
            data_json = json.loads(response.text)
            # resp = requests.get('https://' + content.attrs['href'].strip('//'))
            # soup2 = BeautifulSoup(resp.text,'html.parser')
            
            df = pd.DataFrame(
                data = [{
                    '公司名稱':data_json['data']['header']['custName'],
                    '公司鏈結':data_json['data']['header']['custUrl'],#company_url
                    '具備證照':data_json['data']['condition']['certificate'],
                    '工作職稱':data_json['data']['header']['jobName'],
                    '工作內容':data_json['data']['jobDetail']['jobDescription'],
                    '職務類別':[x['description'] for x in data_json['data']['jobDetail']['jobCategory']],
                    '工作待遇':data_json['data']['jobDetail']['salary'],
                    '工作性質':job_type_list[int(data_json['data']['jobDetail']['hireType'])],
                    '上班地點':data_json['data']['jobDetail']['addressRegion']+data_json['data']['jobDetail']['addressDetail'],
                    '管理責任':data_json['data']['jobDetail']['manageResp'],
                    '出差外派':data_json['data']['jobDetail']['businessTrip'],
                    '上班時段':data_json['data']['jobDetail']['workPeriod'],
                    '休假制度':data_json['data']['jobDetail']['vacationPolicy'],
                    '可上班日':data_json['data']['jobDetail']['startWorkingDay'],
                    '需求人數':data_json['data']['jobDetail']['needEmp'],
                    '接受身份':[x['description'] for x in data_json['data']['condition']['acceptRole']['role']],
                    '學歷要求':data_json['data']['condition']['edu'],
                    '工作經歷':data_json['data']['condition']['workExp'],
                    '語文條件':data_json['data']['condition']['language'],
                    '工作技能':data_json['data']['condition']['skill'],
                    '其他條件':data_json['data']['condition']['other'],
                    '公司福利':data_json['data']['welfare']['welfare'],
                    '科系要求':data_json['data']['condition']['major'],
                    '聯絡方式':data_json['data']['contact'],
                    '連結路徑':"https://www.104.com.tw/job/%s"%job_num}],
                columns = ['公司名稱','工作職稱','工作內容','職務類別','工作待遇','工作性質','上班地點','管理責任','出差外派',
                            '上班時段','休假制度','可上班日','需求人數','接受身份','學歷要求','工作經歷','語文條件',
                            '工作技能','其他條件','公司福利','科系要求','聯絡方式','連結路徑','具備證照','公司標記','公司照片',
                        '公司鏈結','擅長工具'])
            # try:
            #     df['具備證照'] = data_json['data']['condition']['certificate']
            # except:
            #     pass
            try:
                df['公司標記'] = data_json['data']['custLogo']#company_icon
            except:
                pass
            try:
                df['公司照片'] = data_json['data']['environmentPic']['corpImageBottom']['imageUrl']
            except:
                pass
            try:
                df['擅長工具'] = [x['description'] for x in data_json['data']['jobDetail']['specialty']]
            except:
                pass
            
            JobList = JobList.append(df, ignore_index=True)
            i += 1
            print("Success and Crawl Next 目前正在爬第" + str(i) + "個職缺資訊")
            time.sleep(0.5) # 執行完休息0.5秒，避免造成對方主機負擔
            # except Exception as e:
            #     print(e)
            #     print(content)
            #     print("Fail and Try Again!")
        except Exception as e:
            print(e)
    threading.Thread(target=make_data_into_excel, args=(JobList,sheetName)).start() 
    return JobList
    
def make_data_into_excel(JobList,sheetName):
    job_data_frames = JobList
    if (os.path.exists('./jobs.xlsx')):
        try:
            print("jobs.xlsx exit!! Trying Write")
            # with open("./jobs.xlsx", "r") as f:
            #     book = load_workbook('jobs.xlsx')
            #     writer = pd.ExcelWriter('jobs.xlsx', engine='openpyxl')
                # writer.book = book
                # writer.sheets = {ws.title: ws for ws in book.worksheets}

            job_data_frames_old = pd.read_excel('jobs.xlsx',sheet_name = sheetName)
            print(job_data_frames_old)
            new_job_data_frames = job_data_frames_old.append(job_data_frames,ignore_index = True,sort = False)
            print(new_job_data_frames)
            new_job_data_frames.drop_duplicates(subset=['公司名稱', '工作職稱', '工作內容'],inplace = True)
            print(new_job_data_frames)
            new_job_data_frames.to_excel('./jobs.xlsx',sheet_name = sheetName, index = False)
            # writer.save()
            print("Write Done")
        except Exception as e:
            print(e)
            # writer.save()
    else:
        job_data_frames.to_excel('./jobs.xlsx',sheet_name = sheetName, index = False)
        

In [ ]:
# #manga_UrlKey = 'manhua-beifanchangguideyingxiongyangyudade-changshiwaidemofajianshi' (e.g.) 查看漫畫的集數
# def Manga_chapterList(manga_UrlKey):
# #         manga_index = 'http://www.manhuaren.com/%s/?from=/manhua-list-area36-s2/'%manga_UrlKey
#     manga_index = 'http://www.manhuaren.com/%s/'%manga_UrlKey
#     user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
#     headers = { 'User-Agent' : user_agent }
#     index = requests.get(manga_index,headers=headers, verify = False)
#     soup = BeautifulSoup(index.text,"lxml")
#     chapterRawLists = soup.find_all('a',class_='chapteritem')
#     chapter_list = {}
#     for i in chapterRawLists:
#         chapter_list[i.text] = i.get_attribute_list('href')
#     return chapter_list

In [ ]:
# #manga = {'ShowPicUrlB':'__圖片網址__','ID':數字}
# def Manga_make_jpg(manga):
#     img_url = manga['ShowPicUrlB']
#     try:
#         os.mkdir('./photo')
#     except:
#         pass
#     if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
#         pass
#     else:
#         res = requests.get(img_url,verify = False).content
#         with open('./photo/%s.jpg'%str(manga['ID']),'wb') as f:     #二進位制寫入
#             f.write(res)

In [ ]:
# def search_manga_info(count,search_text):
#     Web_Site_URL = "https://www.manhuaren.com/pagerdata.ashx?"
#     user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
#     headers = { 'User-Agent' : user_agent }
#     form_data = {
#         't':'7',
#         'pageindex':count,
#         'f':'0',
#         'title':search_text}
#     data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
#     new_data_dict = {}
#     for i in data_dict:
#         if('UpdateComicItems' in new_data_dict):
#             new_data_dict['UpdateComicItems'].append({'ID':i['Id'],'Title':i['Title'],'UrlKey':i['Url'],'ShowPicUrlB':i['BigPic'],})
#         else:
#             new_data_dict['UpdateComicItems']= []
#             new_data_dict['UpdateComicItems'].append({'ID':i['Id'],'Title':i['Title'],'UrlKey':i['Url'],'ShowPicUrlB':i['BigPic'],})
#     for manga in new_data_dict['UpdateComicItems']:
#         threading.Thread(target = Manga_make_jpg,args = (manga,)).start()
#     return new_data_dict

In [ ]:
def get_setting_Message(userID):
    config = configparser.ConfigParser()
    config.read(('./personal_config/%s.ini')%(userID))
    check_section(('./personal_config/%s.ini')%(userID))
    with open("./LineBotJsonFile/job_setting_list.json", encoding="utf8") as json_file:
        # loads=>把文字轉換成物件(字串跟字典組成)  // dumps=>把物件轉成文字
        json_str = json_file.read()
    if ('keyword' in config['params']):
        keyword = config['params']['keyword']
        if (keyword == ""):
            keyword = '目前未設定'
            # if ('、' in keyword):
            #     keyword = keyword.replace('、','%20')
        # else:
        #     keyword = 
    else:
        keyword = '目前未設定'
    if ('area' in config['params']):
        area = config['params']['area']
        if (area != ""):
            # if ('%%2C' in area):
            #     area = area.replace('%2C','、')
            area_encode={'台北市':'600100100',
                '新北市':'6001002000',
                '宜蘭縣':'6001003000',
                '基隆市':'6001004000',
                '桃園市':'6001005000',
                '新竹縣市':'6001006000',
                '苗栗縣':'6001007000',
                '台中市':'6001008000',
                '彰化縣':'6001010000',
                '南投縣':'6001011000',
                '雲林縣':'6001012000',  
                '嘉義縣市':'6001013000',  
                '台南市':'6001014000',  
                '高雄市':'6001016000',  
                '屏東縣':'6001018000',  
                '台東縣':'6001019000',  
                '花蓮縣':'6001020000',  
                '澎湖縣':'6001021000',  
                '金門縣':'6001022000',  
                '連江縣':'6001020000',}
            for ctName in area_encode:
                if (area_encode[ctName] in area):
                    area = area.replace(area_encode[ctName],ctName)
        else:
            area = '全部'
    else:
        area = '全部'
    if ('ro' in config['params']):
        ro = config['params']['ro']
        job_type_decode = ['全部','全職','兼職','高階','派遣','接案','家教','其他']
        ro = job_type_decode[1]
    if ('sctp' in config['params']):  
        sctp = config['params']['sctp']
        scmin = config['params']['scmin']
        if (sctp != ''):
            if (sctp =='H'):
                sctp = '時薪'
            else:
                sctp = '月薪'
        else:
            sctp = '全部'
            scmin = ''
    else:
        sctp = '全部'
        scmin = ''
    json_str = json_str.replace('__關鍵字__',keyword)
    json_str = json_str.replace('__地區__',area)
    json_str = json_str.replace('__薪資__',str(sctp+scmin))
    json_str = json_str.replace('__工作性質__',ro)
    obj = json.loads(json_str)
    print(obj)
    return FlexSendMessage.new_from_json_dict(obj)

In [ ]:
NotFoundPage = 'https://www.104.com.tw/jobs/main/syserr?eid=14805901391537855361'
def make_flex_message(df):
    job_listObject_Json = json.load(open('LineBotJsonFile/job_ListObject.json',))
    
    for i in range(9):
        with open("./LineBotJsonFile/job_list_Message.json", encoding="utf8") as json_file:
            # loads=>把文字轉換成物件(字串跟字典組成)  // dumps=>把物件轉成文字
            json_str = json_file.read()
        # job_item_Message_Json = json.load(open('LineBotJsonFile/job_list_Message.json',))
        # df.values[i][0]#

        json_str = json_str.replace('工作名稱',str(df['工作職稱'].values[i]))
        json_str = json_str.replace('公司名稱',str(df['公司名稱'].values[i]))
        if (str(df['公司鏈結'].values[i]) == '' ):
            json_str = json_str.replace('https://www.company_url.com/',str(NotFoundPage))
        else:
            json_str = json_str.replace('https://www.company_url.com/',str(df['公司鏈結'].values[i]))
        if (str(df['公司標記'].values[i]) == ''):
            json_str = json_str.replace('https://www.company_icon.com/',str(NotFoundPage))
        else:
            json_str = json_str.replace('https://www.company_icon.com/',str(df['公司標記'].values[i]))
        json_str = json_str.replace('工作待遇',"工作待遇："+str(df['工作待遇'].values[i]))
        json_str = json_str.replace('工作性質',"工作性質："+str(df['工作性質'].values[i]))
        json_str = json_str.replace('休假制度',"休假制度："+str(df['休假制度'].values[i]))
        json_str = json_str.replace('需求人數',"需求人數"+str(df['需求人數'].values[i]))
        json_str = json_str.replace('工作性質',"工作性質："+str(df['工作性質'].values[i]))
        if (str(df['公司照片'].values[i]) == ''):
            json_str = json_str.replace('https://www.company_envImg.com/',str(NotFoundPage))
        else:
            json_str = json_str.replace('https://www.company_envImg.com/',str(df['公司照片'].values[i]))
        json_str = json_str.replace('https://Job_Url.com',str(df['連結路徑'].values[i]))
        obj = json.loads(json_str)
        job_listObject_Json['contents']['contents'].append(obj)
    

    job_seeMore_Message_Json = json.load(open('LineBotJsonFile/job_seeMore.json',))
    job_listObject_Json['contents']['contents'].append(job_seeMore_Message_Json)
    print(job_listObject_Json)
    return FlexSendMessage.new_from_json_dict(job_listObject_Json)

def check_section(file_name):#確定section有沒有缺少
    config = configparser.ConfigParser()
    config.read(file_name)
    if (config.has_section("params") == False):
        config.add_section("params")
        config.set("params","ro","0")
        config.set("params","keyword","")
        config.set("params","isnew","30")
        config.set("params","mode","s")
        config.set("params","area","")
        config.set("params","asc","0")
        config.set("params","order","14")
        config.set("params","jobsource","2018indexpoc")
        # config.write(open((file_name, 'w')))
    if (config.has_section("personstatus") == False):
        config.add_section("personstatus")
        config.set("personstatus","catch_word","0")
        # config.write(open(file_name, 'w'))
    with open(file_name, 'w') as configfile:
        config.write(configfile)
# {'ro': '0', 'keyword': 'Python', 'isnew': '30', 'mode': 's', 'area': '6001002000%2C600100100', 'asc': '0', 'order': '14', 'page': 1,'jobsource':'2018indexpoc'}
def handle_params(params,userID):#將params內容寫入個人設定檔
    config = configparser.ConfigParser()
    config.read(('./personal_config/%s.ini')%(userID))
    check_section(('./personal_config/%s.ini')%(userID))
    for key in params:
        config.set('params', key, str(params[key]))
    with open(('./personal_config/%s.ini')%(userID), 'w') as configfile:
        config.write(configfile)
    # config.write(open(('./personal_config/%s.ini')%(userID), 'w'))

    # {'ro': '0', 'order': '11', 'asc': '0', 'page': '1', 'jobexp': '1', 'mode': 's', 'jobsource': '2018indexpoc'}

In [ ]:
def get_Menu_Message():
    buttons_template_message = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='https://i2.wp.com/blog.104.com.tw/wp-content/uploads/2018/02/104logo-orange.png?',
                title='Menu',
                text='Please select',
                actions=[
                    PostbackAction(
                        label='新鮮人推薦工作',
                        display_text='新鮮人找工作',
                        data='lmode=1&ro=0&order=11&asc=0&page=1&jobexp=1&mode=s&jobsource=2018indexpoc'
                    ),
                    #以用戶身份發送text
                    MessageAction(
                        label='找工作',
                        text='設定',

                    ),
                    #網址鏈結 參考：google:'line URI scheme'
                    URIAction(
                        label='透過位置尋找',
                        uri='https://line.me/R/nv/location/'),
                    
                    #打電話
                    URIAction(
                        label='聯絡客服',
                        uri='tel://113'
                    )
                ]
            )
        )
    return buttons_template_message

In [ ]:
def transfer_params(params):
    for key in params:
        if ("、" in str(params[key])):
            if (key == 'keyword'):
                params[key] = params[key].replace("、","%20")
            if (key == 'area'):
                params[key] = params[key].replace("、","%2C")
    return params

In [ ]:
def makeMessageReply(reply_token,params):
    params = transfer_params(params)
    df = Catch_104_joblist(my_paramas = params)
    df = df.fillna('https://www.104.com.tw/jobs/main/syserr?eid=14805901391537855361')
    jobListLineMessage = make_flex_message(df)
    # print(event.reply_token)
    line_bot_api.reply_message(
        reply_token,
        jobListLineMessage)

In [ ]:
@app.route("/",methods=['GET'])
def getdata():
	return "Hello Workd"

In [ ]:
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

Text Message Event

In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    config = configparser.ConfigParser()
    config.read(('./personal_config/%s.ini')%(event.source.user_id))
    print(event)
    # personal_setting_Message = get_setting_Message(event.source.user_id)
    check_section(('./personal_config/%s.ini')%(event.source.user_id))
    # if (config.has_section("params") == False):
    #     config.add_section("params")
        # with open(('./personal_config/%s.ini')%(event.source.user_id), 'w') as configfile:
        #     config.write(configfile)
    if (event.message.text == '::找工作::'):
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage("請輸入關鍵字(例：工作職稱、公司名、技能專長...):")
            )
        config.set('personstatus','catch_word',str(1))
        with open(('./personal_config/%s.ini')%(event.source.user_id), 'w') as configfile:
            config.write(configfile)
        # config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        
    elif (event.message.text == '目錄'):
        line_bot_api.reply_message(
            event.reply_token,
            get_Menu_Message()
            )
    elif (event.message.text == '設定'):
        line_bot_api.reply_message(
            event.reply_token,
            get_setting_Message(event.source.user_id)
            )
    elif (event.message.text == '添加'):
        pass
    elif (event.message.text == '::查看更多::'):
        # with open(('./personal_config/%s.ini')%(event.source.user_id) 'w'):
        page = int(config['params']['page'])+1
        config.set('params','page',str(page))
        with open(('./personal_config/%s.ini')%(event.source.user_id), 'w') as configfile:
            config.write(configfile)
        # config.write(open(('./personal_config/%s.ini')%(event.source.user_id)), 'w')
        params = {}
        # for section in config['']:
        #     # print(i)
        for key in config['params']:
            
            params[key] = config['params'][key]
            # if (params[key])
        # print(params)
        threading.Thread(target = makeMessageReply,args = (event.reply_token,params)).start()
        # params = transfer_params(params)
        # print(params)
        # df = Catch_104_joblist(my_paramas = params)
        # df = df.fillna('https://www.104.com.tw/jobs/main/syserr?eid=14805901391537855361')
        # jobListLineMessage = make_flex_message(df)
        # # print(event.reply_token)
        # line_bot_api.reply_message(
        #     event.reply_token,
        #     jobListLineMessage)
    else:
        if (config['personstatus']['catch_word'] == str(1)):
            if ('keyword' not in config['params']):
                config.set('params','keyword',str(event.message.text))
            elif (config['params']['keyword'] != ''):
                config.set('params','keyword',config['params']['keyword']+"、"+str(event.message.text))
            else:
                config.set('params','keyword',str(event.message.text))
            config.set('personstatus','catch_word',str(0))
            with open(('./personal_config/%s.ini')%(event.source.user_id), 'w') as configfile:
                config.write(configfile)
            # config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            line_bot_api.reply_message(
                event.reply_token,
                [TextSendMessage("已經添加關鍵字:"+event.message.text),get_setting_Message(event.source.user_id)]
                )

In [ ]:
@handler.add(MessageEvent, message=LocationMessage)
def handle_message(event):
    print(event)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,[TextSendMessage("%s您好，歡迎使用本服務，目前暫不提供透過地理查詢:"%user_profile.display_name),])

Image Message Event

In [ ]:
@handler.add(MessageEvent, message=ImageMessage)
def handle_ImgMessage(event):
    print(event)
    #line_bot_api get_content (抓賴訊息傳的圖)
    ImgMessage_content = line_bot_api.get_message_content(event.message.id)
    print(ImgMessage_content)
    
    with open('./%s.png'%str(event.message.id),'wb') as f:     #二進位制寫入
        for chunk in ImgMessage_content.iter_content():
            f.write(chunk)
    
    #上傳到S3
    # response = s3_client.upload_file(('./%s.png'%str(event.message.id)), 'iii-tutorial-v2', ('Student9487/%s.png'%str(event.message.id)))
    
    line_bot_api.reply_message(event.reply_token,[TextSendMessage("你傳的圖片被我抓了哈哈哈"), ImageSendMessage(
        original_content_url = 'https://stickershop.line-scdn.net/stickershop/v1/product/1052655/LINEStorePC/main.png;compress=true',
        preview_image_url= 'https://stickershop.line-scdn.net/stickershop/v1/product/1052655/LINEStorePC/main.png;compress=true')])
    # line_bot_api.reply_message(
    #     event.reply_token,
    #     TextSendMessage(text=event.message.text)

Video Message Event

In [ ]:
@handler.add(MessageEvent, message=VideoMessage)
def handle_Mp4Message(event):
    print(event)
    ImgMessage_content = line_bot_api.get_message_content(event.message.id)
    print(ImgMessage_content)
    with open('./%s.mkv'%str(event.message.id),'wb') as f:     #二進位制寫入
        for chunk in ImgMessage_content.iter_content():
            f.write(chunk)
    line_bot_api.reply_message(event.reply_token,[TextSendMessage("你傳的影片被我抓了哈哈哈"), ImageSendMessage(
        original_content_url = 'https://stickershop.line-scdn.net/stickershop/v1/product/1052655/LINEStorePC/main.png;compress=true',
        preview_image_url= 'https://stickershop.line-scdn.net/stickershop/v1/product/1052655/LINEStorePC/main.png;compress=true')])
    # line_bot_api.reply_message(
    #     event.reply_token,
    #     TextSendMessage(text=event.message.text))
        

Postback Event

In [ ]:
# p = {'ro': '0', 'order': '11', 'asc': '0', 'page': '1', 'jobexp': '1', 'mode': 's', 'jobsource': '2018indexpoc'}
# for i in p:
#     print(i)
# 'ro' in config['params']

In [ ]:
@handler.add(PostbackEvent)
def handle_Postback(event):
    print(event)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    data_str = event.postback.data
    config = configparser.ConfigParser()
    config.read(('./personal_config/%s.ini')%(event.source.user_id))
    data_split = data_str.split('&')
    Mode = -1
    params = {}
    # check_section(('./personal_config/%s.ini')%(event.source.user_id))
    
    for item in data_split:
        if (item.split('=')[0]=='lmode'):
            Mode = item.split('=')[1]
        else:
            params[item.split('=')[0]] = item.split('=')[1]
    print("Mode = :",Mode)
    if Mode == '1':#新鮮人找工作
        print(params)
        params['page'] = 1
        handle_params(params,event.source.user_id)
        threading.Thread(target = makeMessageReply,args = (event.reply_token,params)).start()
        # df = Catch_104_joblist(my_paramas = params)
        # df = df.fillna('https://www.104.com.tw/jobs/main/syserr?eid=14805901391537855361')
        # jobListLineMessage = make_flex_message(df)
        # config.set('params','page',str('1'))
        # config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        # # print(event.reply_token)
        # line_bot_api.reply_message(
        #     event.reply_token,
        #     jobListLineMessage)
    elif Mode == '3':#設定搜尋條件
        if ('area' in params):
            if (params['area'] == "North"):
                text_message = TextSendMessage(text='請選擇地點',
                                quick_reply=QuickReply(items=[
                                    QuickReplyButton(action=PostbackAction(label="臺北市", display_text="臺北市",data='lmode=4&area=600100100')),
                                    QuickReplyButton(action=PostbackAction(label="新北市", display_text="新北市",data='lmode=4&area=6001002000')),
                                    QuickReplyButton(action=PostbackAction(label="基隆市", display_text="基隆市",data='lmode=4&area=6001004000')),
                                    QuickReplyButton(action=PostbackAction(label="桃園市", display_text="桃園市",data='lmode=4&area=6001005000')),
                                    QuickReplyButton(action=PostbackAction(label="新竹縣市", display_text="新竹縣市",data='lmode=4&area=6001006000')),
                                    QuickReplyButton(action=PostbackAction(label="苗栗縣", display_text="苗栗縣",data='lmode=4&area=6001007000')),
                                ]))
                line_bot_api.reply_message(
                    event.reply_token,
                    text_message
                    )
            if (params['area'] == "central"):
                text_message = TextSendMessage(text='請選擇地點',
                                quick_reply=QuickReply(items=[
                                    QuickReplyButton(action=PostbackAction(label="宜蘭縣", display_text="宜蘭縣",data='lmode=4&area=6001003000')),
                                    QuickReplyButton(action=PostbackAction(label="花蓮縣", display_text="花蓮縣",data='lmode=4&area=6001020000')),
                                    QuickReplyButton(action=PostbackAction(label="臺東縣", display_text="臺東縣",data='lmode=4&area=6001019000')),
                                    QuickReplyButton(action=PostbackAction(label="臺中市", display_text="臺中市",data='lmode=4&area=6001008000')),
                                    QuickReplyButton(action=PostbackAction(label="彰化縣", display_text="彰化縣",data='lmode=4&area=6001010000')),
                                    QuickReplyButton(action=PostbackAction(label="雲林縣", display_text="雲林縣",data='lmode=4&area=6001012000')),
                                    QuickReplyButton(action=PostbackAction(label="南投縣", display_text="南投縣",data='lmode=4&area=6001011000')),
                                ]))
                line_bot_api.reply_message(
                    event.reply_token,
                    text_message
                    )
            if (params['area'] == "south"):
                text_message = TextSendMessage(text='請選擇地點',
                                quick_reply=QuickReply(items=[
                                    QuickReplyButton(action=PostbackAction(label="嘉義縣", display_text="嘉義縣",data='lmode=4&area=6001013000')),
                                    QuickReplyButton(action=PostbackAction(label="臺南市", display_text="臺南市",data='lmode=4&area=6001014000')),
                                    QuickReplyButton(action=PostbackAction(label="高雄市", display_text="高雄市",data='lmode=4&area=6001016000')),
                                    QuickReplyButton(action=PostbackAction(label="屏東縣", display_text="屏東縣",data='lmode=4&area=6001018000')),
                                    QuickReplyButton(action=PostbackAction(label="澎湖縣", display_text="澎湖縣",data='lmode=4&area=6001021000')),
                                    QuickReplyButton(action=PostbackAction(label="金門縣", display_text="金門縣",data='lmode=4&area=6001022000')),
                                    QuickReplyButton(action=PostbackAction(label="連江縣", display_text="連江縣",data='lmode=4&area=6001020000')),
                                ]))
                line_bot_api.reply_message(
                    event.reply_token,
                    text_message
                    )
            if (params['area'] == "all"):
                # config = configparser.ConfigParser()
                # config.read(('./personal_config/%s.ini')%(event.source.user_id))
                config.set('params','area',str(''))
                config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            
        if ('keyword' in params):
            if (params['keyword'] == 'reset'):
                # config = configparser.ConfigParser()
                # config.read(('./personal_config/%s.ini')%(event.source.user_id))
                config.set('params','keyword',str(''))
                config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
                line_bot_api.reply_message(
                    event.reply_token,
                    [TextSendMessage("已為您重設關鍵字"),get_setting_Message(event.source.user_id)]
                    )
                config.set('personstatus','catch_word',str(0))
                config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage("請輸入關鍵字(例：工作職稱、公司名、技能專長...):")
                    )
                config.set('personstatus','catch_word',str(1))
                config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        if ('ro' in params):
            # config = configparser.ConfigParser()
            # config.read(('./personal_config/%s.ini')%(event.source.user_id))
            config.set('params','ro',str(params['ro']))
            config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("已為您設定")
                )
        if ('sctp' in params):
            # config = configparser.ConfigParser()
            # config.read(('./personal_config/%s.ini')%(event.source.user_id))
            config.set('params','sctp',str(params['sctp']))
            config.set('params','scmin',str(params['scmin']))
            config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("已為您設定")
                )
            # config.set('personstatus','catch_word',str(0))
            # config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
            

    elif Mode == '2':#搜尋工作

        
        # config = configparser.ConfigParser()
        # config.read(('./personal_config/%s.ini')%(event.source.user_id))
        # config.set('params','page',str(int(config['params']['page'])+1))
        # config.write(open(('./personal_config/%s.ini')%(event.source.user_id)), 'w')
        params = {}
        # for section in config:
        #     # print(i)
        #     for key in config[section]:
        #         params[section] = config[section][key]
        for key in config['params']:
            params[key] = config['params'][key]
        # print("params = ",params)
        # params = transfer_params(params)
        params['page'] = 1
        config.set('params','page',str('1'))
        config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        threading.Thread(target = makeMessageReply,args = (event.reply_token,params)).start()
        print("transfer_params = ",params)
        # df = Catch_104_joblist(my_paramas = params)
        # df = df.fillna('https://www.104.com.tw/jobs/main/syserr?eid=14805901391537855361')
        # jobListLineMessage = make_flex_message(df)
        # # print(event.reply_token)
        # line_bot_api.reply_message(
        #     event.reply_token,
        #     jobListLineMessage)
    elif Mode == '4': #QuickReply設定地區
        
        # config = configparser.ConfigParser()
        # config.read(('./personal_config/%s.ini')%(event.source.user_id))
        if (config['params']['area'] != ''):
            config.set('params','area',config['params']['area']+"、"+str(params['area']))
            config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        else:
            config.set('params','area',str(params['area']))
            config.write(open(('./personal_config/%s.ini')%(event.source.user_id), 'w'))
        # personal_setting_Message = get_setting_Message(event.source.user_id)
        line_bot_api.reply_message(
                    event.reply_token,
                    [TextSendMessage("已為您添加"),get_setting_Message(event.source.user_id)]
                    )
    elif Mode == '5':
        path = ('./personal_config/%s.ini')%(event.source.user_id)
        os.remove(path)
        check_section(('./personal_config/%s.ini')%(event.source.user_id))
        line_bot_api.reply_message(
                    event.reply_token,
                    [TextSendMessage("已為您重設"),get_setting_Message(event.source.user_id)]
                    )
    else :#未知
        pass


# 公司名稱,工作職稱,工作內容,職務類別,工作待遇,工作性質,上班地點,管理責任,出差外派,上班時段,休假制度,可上班日,需求人數,接受身份,學歷要求,工作經歷語文條件,工作技能
# ,其他條件,公司福利,科系要求,聯絡方式,連結路徑,公司標記

In [ ]:
# params = {'ro': '0', 'order': '11', 'asc': '0', 'page': '1', 'jobexp': '1', 'mode': 's', 'jobsource': '2018indexpoc'}
# Catch_104_joblist(my_paramas = params)

In [ ]:
@handler.add(UnfollowEvent)
def handle_unfollow_event(event):
    print(event)
    # user_profile = line_bot_api.get_profile(event.source.user_id)
    # 把用戶個資打印出來
    # print(user_profile.user_id)
    # print(user_profile.display_name)
    # print(user_profile.picture_url)
    # print(user_profile.status_message)
    path = ('./personal_config/%s.ini')%(event.source.user_id)
    os.remove(path)
    line_bot_api.unlink_rich_menu_from_users([event.source.user_id])

Follow Event

In [ ]:
@handler.add(FollowEvent)
def handle_follow_event(event):
    print(event)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    # 把用戶個資打印出來
    print(user_profile.user_id)
    print(user_profile.display_name)
    print(user_profile.picture_url)
    print(user_profile.status_message)
    FileName = ('./personal_config/%s.ini')%(event.source.user_id)
    # threading.threadint(check_section())
    # threading.Thread(target=check_section,args=[app,]).start()
    line_bot_api.link_rich_menu_to_users([user_profile.user_id],RichMenuId)
    threading.Thread(target = check_section,args = (FileName,)).start()
    with open("./personal_info/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')

    res = requests.get(user_profile.picture_url,verify = False).content
    with open('./personal_info/%s.jpg'%str(user_profile.display_name),'wb') as f:     #二進位制寫入
        f.write(res)
    

    line_bot_api.reply_message(
        event.reply_token,[TextSendMessage("%s您好，歡迎使用本服務，請選擇服務項目:"%user_profile.display_name),
        get_Menu_Message()])


In [ ]:
#使用ngrok套件
# import socket
# print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))
run_with_ngrok(app)

In [ ]:
'''

啟動Server

'''

# def run_flask_app(app):
app.run()
# threading.Thread(target=run_flask_app,args=[app,]).start() 

In [ ]:
# config = configparser.ConfigParser()
# config.read(('./personal_config/%s.ini')%('U2c0fd74c8005427505431b35a67571ab'))

# if(config.has_section("device_time") == False):
    # config.add_section("device_time")
    # config.write(open(('./personal_config/%s.ini')%('U2c0fd74c8005427505431b35a67571ab'), 'w'))

In [ ]:
# a= '6001001000%2C6001004000%20python'
# a.replace("%2C","、")

In [ ]:
# import ast
# for i in config:
#     print(i)
#     for j in config[i]:
#         print(j)
#         print(config[i][j])
#         print(config[i][j] =='')
#         print(type(config[i][j]))
#         # print(ast.literal_eval(config[i][j]))